# Smart Facility: Spills Basic Dataset - Demo Notebook

Demo 1: Filtering/querying the dataset based on metadata.  
Demo 2: Visualizing labels for a specific video.

Note: this notebook relies on the `infinity-tools` python module, which can be installed following instructions [here](https://github.com/toinfinityai/infinity-tools#installation).

In [1]:
import os
import glob
import json
import shutil
import pandas as pd
from pycocotools.coco import COCO
from IPython.display import clear_output
from infinity_tools.spills import vis
from infinity_tools.common.vis.videos import parse_video_frames, stack_videos

## Demo 1: Dataset Filtering

We first show how to load the dataset's metadata into an easily filtered dataframe.

In [2]:
PATH_TO_DATASET_FOLDER = "PATH_TO_DATASET_FOLDER"

In [3]:
all_param_paths = sorted(glob.glob(os.path.join(PATH_TO_DATASET_FOLDER, "*_params.json")))

metadata = []
for param_path in all_param_paths:
    metadata_dict = {}
    metadata_dict["video_number"] = os.path.basename(param_path).split("_")[0]
    parameters = json.load(open(param_path))["params"]
    metadata_dict.update(parameters)
    metadata.append(metadata_dict)

metadata = pd.DataFrame(metadata)
metadata.head()

,video_number,scene,color,size,aspect_ratio,profile_irregularity,depth,frame_rate,video_duration,random_seed
0,00000,WAREHOUSE_0002,red,62.547932,1.693397,0.424591,1.411789,24,6.377851,1916477150
1,00001,WAREHOUSE_0004,transparent,100.000000,1.731358,0.458503,1.118880,24,10.997811,465231991
2,00002,HOSPITAL_0001,dark_green,46.094629,1.842057,0.178404,1.456069,24,8.913092,513287503
3,00003,HOSPITAL_0002,transparent,37.981807,1.494779,0.211677,1.287602,24,6.953701,509504814
4,00004,GROCERY_0001,transparent,11.521043,2.011785,0.441531,1.996272,24,6.300116,1560628111


You can now filter and/or query the dataset for specific properties, as shown in the example below.

In [4]:
filtered_samples = metadata.query("scene == 'HOSPITAL_0002'").query('video_duration < 6')
display(filtered_samples.head())

,video_number,scene,color,size,aspect_ratio,profile_irregularity,depth,frame_rate,video_duration,random_seed
12,00012,HOSPITAL_0002,transparent,55.204445,1.755085,0.474321,1.618665,24,5.903868,600965452
29,00029,HOSPITAL_0002,transparent,71.482705,1.877697,0.000000,1.769881,24,5.716090,1557554297


## Demo 2: Label Visualization

We next show how to visualize labels for a specific spills video. We do this using the dataset sample provided in the repo.

In [5]:
def visualize_all_labels(video_rgb_path: str) -> str:
    """Visualizes RGB, bounding boxes and segmentation masks in single video."""

    output_directory = os.path.dirname(video_rgb_path)
    labels_path = video_rgb_path.replace(".mp4", "_labels.json")
    seg_zip_path = video_rgb_path.replace(".mp4", "_segmentation.zip")
    
    seg_extracted_path = seg_zip_path.replace(".zip", "")
    os.makedirs(seg_extracted_path, exist_ok=True)
    vis.unzip(seg_zip_path, seg_extracted_path)

    imgs = parse_video_frames(video_rgb_path)
    image_dims = (imgs.shape[2], imgs.shape[1])
    fps = 24
    coco = COCO(labels_path)

    bounding_box_path = vis.create_bounding_boxes_video(
        os.path.join(output_directory, "bounding_box.mp4"), imgs, fps, coco, image_dims
    )
    segmentation_path = vis.create_segmentation_video(
        os.path.join(output_directory, "segmentation.mp4"),
        seg_extracted_path,
        fps,
        image_dims,
    )

    merged_video_path = stack_videos(
        paths=[video_rgb_path, bounding_box_path, segmentation_path], 
        axis=2,
        output_path=os.path.join(output_directory, "labels.mp4")
    )
    
    remove_paths = [bounding_box_path, segmentation_path]
    for e in remove_paths:
        os.remove(e)
    shutil.rmtree(seg_extracted_path)
    clear_output()

    return merged_video_path

In [6]:
video_rgb_path = "./sample_data/00000.mp4"
labels_video_path = visualize_all_labels(video_rgb_path)
print(f"The resulting labels video can be viewed at: {labels_video_path}")

The resulting labels video can be viewed at: ./sample_data/labels.mp4
